# Objetivo

Avaliar quais das 180 features são mais importantes.

# Método

Aplicar _pandas_profiling_ para verificar features com muitos NaN (*not a number*) e correlação alta com outras features.

In [2]:
# imports
import pandas as pd
import feather
import pandas_profiling

# Carregamento do arquivo de teste com 10 mil amostras

## Google Colab

Se estiver usando Google Colab, executar o código a seguir, para acessar o arquivo de dados no Drive.

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls 'drive/My Drive/Colab Notebooks'

In [7]:
os.path.isfile("drive/My Drive/Colab Notebooks/market_sample10k.feather")

True

In [ ]:
market10k = feather.read_dataframe('drive/My Drive/Colab Notebooks/market_sample10k.feather')

## Arquivo local

Se estiver usando arquivo local, simplesmente carregá-lo.

In [8]:
market10k = feather.read_dataframe('market_sample10k.feather')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 182 entries, Unnamed: 0 to qt_filiais
dtypes: bool(9), float64(144), int64(2), object(27)
memory usage: 13.3+ MB


# Geração do Relatório

In [ ]:
market10k.info()

In [0]:
market10k.shape

(10000, 182)

In [0]:
profile10k = pandas_profiling.ProfileReport(market10k)

In [0]:
profile10k.to_file(outputfile='Profile 10k sample from estaticos_market.html')

# Análises

In [ ]:
# Listar features que tem correlação maior que:
threshold_correlation = 0.8

#Listar features que tem fração de missing values (NaN) maior que:
threshold_missing = 0.7

## Features que apresentam alta correlação

In [23]:
# verifica features que tem alta correlação (por exemplo, quantidade de funcionários e funcionários com idade em uma faixa)
rejected_variables_corr = profile10k.get_rejected_variables(threshold = threshold_correlation)
len(rejected_variables_corr)

52

## Features com muitos valores nulos (NaN)

In [0]:
def get_missing_variables(prof, threshold: int = 0.5) -> list:
    """Return a list of variable names being rejected for 
    presenting missing values (NaN).
    
    Args:
        threshold: fraction of NaN values (Default value = 0.5 = 50%)

    Returns:
        A list of missing variables.
    """
    variable_profile = prof.description_set['variables']
    result = []
    for row in variable_profile.index:
      if variable_profile.loc[row]['type']!='CORR':
        if variable_profile.loc[row]['p_missing'] > threshold:
          result.append(row)
    return result

In [22]:
missing_variables = get_missing_variables(profile10k, threshold = threshold_missing)
len(missing_variables)

81

# Conclusão

Em `exclude_variables`, temos o resultado final: uma lista de features a excluir por conta de alta correlação com outras features ou por apresentar grande número de valores NaN.

Neste caso, usando `threshold_correlation = 0.8` e `threshold_missing = 0.7`, podemos excluir **133** features.

In [26]:
# conferir se não há sobreposição nas duas listas 
exclude_variables = list(set(rejected_variables_corr + missing_variables))
len(exclude_variables)

133